In [73]:
import glob
import cv2
import numpy as np

# a)

In [74]:
fire_images = glob.glob("./fire_dataset/fire_images/*")
no_fire_images = glob.glob("./fire_dataset/non_fire_images/*")

In [75]:
target_size = (64, 64)

In [76]:
def preprocess_img(filepath):
    
    img = cv2.imread(filepath)
    
    img = cv2.resize(img,target_size)

    img_array = np.array(img)
    
    img_array = img_array / 255.0
    
    return img_array

In [77]:
images = []
labels = []

In [78]:
for path in fire_images:
    
    img_array = preprocess_img(path)
    label = 1
    
    images.append(img_array)
    labels.append(label)

In [79]:
for path in no_fire_images:
    
    img_array = preprocess_img(path)
    label = 0
    
    images.append(img_array)
    labels.append(label)

In [80]:
images = np.array(images)
labels = np.array(labels)

# b)

In [81]:
from sklearn.model_selection import train_test_split


train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# c)

In [82]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

X_train_reshape = train_images.reshape(train_images.shape[0], -1)
X_test_reshape = test_images.reshape(test_images.shape[0], -1)

logreg.fit(X_train_reshape, train_labels)
y_pred = logreg.predict(X_test_reshape)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# d)

In [83]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
f1_score = f1_score(test_labels, y_pred)
conf_mat = confusion_matrix(test_labels, y_pred)


# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1_score}")
print(f"confusion_matrix:\n {conf_mat}")

Accuracy: 0.92
Precision: 0.9113924050632911
Recall: 0.9863013698630136
F1 score: 0.9473684210526315
confusion_matrix:
 [[ 40  14]
 [  2 144]]


# e)

In [84]:
test_probs = logreg.predict_proba(X_test_reshape)[:, 1]

best_accuracy = 0
best_threshold = 0
for threshold in np.linspace(0, 1, 101):
    predicted_labels = (test_probs >= threshold).astype(int)
    accuracy = accuracy_score(test_labels, predicted_labels)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

predicted_labels = (test_probs >= best_threshold).astype(int)

In [85]:
test_accuracy = accuracy_score(test_labels, predicted_labels)
test_precision = precision_score(test_labels, predicted_labels)
test_recall = recall_score(test_labels, predicted_labels)
test_conf_mat = confusion_matrix(test_labels, predicted_labels)

# Print the metrics
print("Best threshold:", best_threshold)
print(f"Accuracy: {test_accuracy}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")
print(f"confusion_matrix:\n {test_conf_mat}")

Best threshold: 0.92
Accuracy: 0.95
Precision: 0.9594594594594594
Recall: 0.9726027397260274
confusion_matrix:
 [[ 48   6]
 [  4 142]]


# f)

In [86]:
import pickle

with open('best_logreg_model.pkl', 'wb') as f:
    pickle.dump(logreg, f)

In [87]:
image_paths = ["./fire_dataset/fire_images/*",
                   "./fire_dataset/non_fire_images/*"]

In [88]:
def predict_img(path,model, threshold):
    img = preprocess_img(path)
    prob = model.predict_proba(img.reshape(1, -1))[:, 1][0]
    label = (prob >= threshold).astype(int)
    return label, prob

In [89]:
with open("best_logreg_model.pkl", "rb") as f:
    best_model = pickle.load(f)

In [90]:
import matplotlib.pyplot as plt

def show_img(path , label, prob):
    img = cv2.imread(path)
    img = cv2.resize(img,(360,360))

    if label == 1:
        text = f"Fire: {prob:.2f}"
        color = (0, 0, 255)  # Red
    else:
        text = f"No Fire: {(1-prob):.2f}"
        color = (0, 255, 0)  # Green

    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return img  

In [91]:
import os

res_path = './result/'

In [92]:
indices = np.random.choice(len(fire_images), size=5)

for index in indices:
    label, prob = predict_img(fire_images[index], best_model, best_threshold)
    img = show_img(fire_images[index],label, prob)
    cv2.imwrite(os.path.join(res_path , f"fire_{index}.jpg"), img)
    cv2.waitKey(0)

In [93]:
indices = np.random.choice(len(no_fire_images), size=5)

for index in indices:
    label, prob = predict_img(no_fire_images[index], best_model, best_threshold)
    img = show_img(no_fire_images[index],label, prob)
    cv2.imwrite(os.path.join(res_path , f"no_fire_{index}.jpg"), img)
    cv2.waitKey(0)